In [ ]:
import healpy
from cora.util import hputil
import numpy as np
from matplotlib import pyplot as plt
from ch_util import ephemeris as ephem, andata
from caput.time import unix_to_skyfield_time
import h5py
import time
from glob import glob

%load_ext autoreload
%autoreload 2

from continuum_beam import *

%matplotlib inline
plt.rcParams.update({'figure.figsize': (16, 12), 'font.size': 16})

# Try fitting beam using Haslam

## Select a quiet region in RA

In [ ]:
# RA range during nighttime in Sept/October in between bright point source
target_RA = (30., 60.)

In [ ]:
# Look at an acquisition I was using for ring maps
acq_path = "/mnt/gong/archive/20180921T004045Z_chimeN2_corr/"
gain_file = "/mnt/recv1/calibration/gain_updates/gain_20180921T050356.984083Z_cyga.h5"

In [ ]:
# get N2 frequencies
with h5py.File(acq_path + "00000000_0000.h5") as fh:
    freq = fh['index_map/freq']['centre']
    start_time = fh['index_map/time']['ctime'][0]
# read gains
with h5py.File(gain_file) as fh:
    gain = fh['gain'][:]
    gain_freq_ind = [ list(fh['index_map/freq']['centre']).index(f) for f in freq ]

In [ ]:
# choose a frequency
f_ind = 9
print freq[f_ind]

In [ ]:
target_time = [ ephem.transit_times(r, start_time) for r in target_RA ]

In [ ]:
datar = andata.CorrReader(sorted(glob(acq_path + "*.h5"))[:18])

In [ ]:
# select time and freq
datar.select_freq_physical((freq[f_ind],))
datar.select_time_range(*target_time)

In [ ]:
# select intracyl baselines for A
prod_sel = np.zeros(256*257/2, dtype=int)
for i in range(256):
    row_l = 256 - i
    start_i = i*256 - i*(i-1)/2
    full_start_i = i*2048 - i*(i-1)/2
    prod_sel[start_i:start_i + row_l] = full_start_i + np.arange(row_l)
datar.prod_sel = prod_sel

In [ ]:
data = datar.read()

In [ ]:
test_prods = np.zeros((256,256))
for p in data.prod:
    test_prods[p['input_a'], p['input_b']] += 1.

In [ ]:
plt.imshow(test_prods)

In [ ]:
# plot autos
p = 0
for i in range(256):
    plt.plot(np.abs(data.vis[0,p,:]))
    p += 256 - i

## Apply gain and exclude flagged inputs

In [ ]:
vis_cal = data.vis[0,:,:] * np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis]

In [ ]:
wgt_cal = data.weight[0,:,:] / np.abs(np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis])**2

In [ ]:
bad_input = np.prod(data.flags['inputs'], axis=1)
print np.sum(bad_input)
bad_input *= gain[gain_freq_ind[f_ind],:256] != 0
print np.sum(bad_input)
bad_prod = np.where(np.outer(bad_input, bad_input)[np.triu_indices(256)] == 0.)[0]
good_prod = np.where(np.outer(bad_input, bad_input)[np.triu_indices(256)] != 0.)[0]

In [ ]:
# plot autos
p = 0
for i in range(256):
    if not p in bad_prod:
        plt.plot(np.abs(vis_cal[p,:]))
    p += 256 - i

In [ ]:
vis_cal = vis_cal[good_prod,:]
wgt_cal = wgt_cal[good_prod,:]

## Try on a single RA

In [ ]:
test_ind = vis_cal.shape[-1] / 2

In [ ]:
test_model = ModelVis(freq=freq[f_ind])

In [ ]:
ns_baselines = ((data.prod['input_b'] % 256).astype(float)
                 - data.prod['input_a'] % 256)
ns_baselines = ns_baselines[good_prod]
test_model.set_baselines(ns_baselines)

In [ ]:
vis_cal[:,time_slice].shape

In [ ]:
num_pix = 256
max_za = 30.
time_slice = slice(test_ind, test_ind+1)
beam_sol = test_model.fit_beam(data.time[time_slice], vis_cal[:,time_slice], wgt_cal[:,time_slice],
                               num_pix, max_za=max_za)

In [ ]:
plt.plot(np.linspace(-max_za, max_za, num_pix), beam_sol)

In [ ]:
test_vis = test_model.get_vis(data.time[time_slice], vis_cal[:,time_slice],
                               num_pix, max_za=max_za)
test_za = np.radians(np.linspace(-max_za, max_za, num_pix))
test_map = np.dot(test_vis[:,0].T, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
plt.plot(test_map)